In [15]:
import importlib
import dfrtokenuniverse.splitter
import dfrtokenuniverse.word_inventory
import dfrtokenuniverse.constantes

In [28]:
importlib.reload(dfrtokenuniverse.splitter)
importlib.reload(dfrtokenuniverse.word_inventory)
importlib.reload(dfrtokenuniverse.constantes)
from dfrtokenuniverse.splitter import TextSplitter
from dfrtokenuniverse.word_inventory import WordInventory
from dfrtokenuniverse.constantes import KDfrNlp
K = KDfrNlp()

In [29]:
splitter = TextSplitter()
invent = WordInventory()
a_dic = {}
for ttkn, n_grama in splitter.split_by_type("Esto es una forma de tokenizar, hay otra ¿no?\nPues esta es una más ¡SEGURO!"):
    invent.normalize_and_count(ttkn, n_grama, a_dic)
print(a_dic)
for ttkn in a_dic:
    for n_grama in a_dic[ttkn]:
        p_ngrama = n_grama.replace("\n", "¬")
        print(f"{K.TTKN_DESC[ttkn]} contiene '{p_ngrama}' con {a_dic[ttkn][n_grama]} apariciones")
del a_dic

{1: {'esto': 1, 'es': 2, 'una': 2, 'forma': 1, 'de': 1, 'tokenizar': 1, 'hay': 1, 'otra': 1, 'no': 1, 'pues': 1, 'esta': 1, 'más': 1, 'seguro': 1}, 2: {' ': 12, ', ': 1, '\n': 1}, 3: {'¿': 1, '?': 1, '¡': 1, '!': 1}}
Letras contiene 'esto' con 1 apariciones
Letras contiene 'es' con 2 apariciones
Letras contiene 'una' con 2 apariciones
Letras contiene 'forma' con 1 apariciones
Letras contiene 'de' con 1 apariciones
Letras contiene 'tokenizar' con 1 apariciones
Letras contiene 'hay' con 1 apariciones
Letras contiene 'otra' con 1 apariciones
Letras contiene 'no' con 1 apariciones
Letras contiene 'pues' con 1 apariciones
Letras contiene 'esta' con 1 apariciones
Letras contiene 'más' con 1 apariciones
Letras contiene 'seguro' con 1 apariciones
Separadores contiene ' ' con 12 apariciones
Separadores contiene ', ' con 1 apariciones
Separadores contiene '¬' con 1 apariciones
Símbolos contiene '¿' con 1 apariciones
Símbolos contiene '?' con 1 apariciones
Símbolos contiene '¡' con 1 apariciones


# Recuperamos texto

In [21]:
import pickle
wiki_path = r"D:\datos\wiki_1909627_entradas.pkl"
print(f"Leyendo desde: {wiki_path}")
with open(wiki_path, "rb") as wiki_file:
    wiki_data = pickle.load(wiki_file)

Leyendo desde: D:\datos\wiki_1909627_entradas.pkl


In [22]:
print(f"Documentos en wiki: {len(wiki_data)}")

Documentos en wiki: 1909627


In [23]:
dic_ngramas = {}

Realizamos varias pasadas para ir acumulando n-gramas

In [33]:
from IPython.display import clear_output
salta  = 200000
limite = 210000
incremento = 10000

In [36]:
cont_entradas = 0
for entrada in wiki_data:
    if cont_entradas > salta:
        for ttkn, n_grama in splitter.split_by_type(entrada):
            invent.normalize_and_count(ttkn, n_grama, dic_ngramas)
        for linea in wiki_data[entrada]:
            for ttkn, n_grama in splitter.split_by_type(linea):
                invent.normalize_and_count(ttkn, n_grama, dic_ngramas)
        if cont_entradas > limite:
            break
        if cont_entradas % 1000 == 0:
            clear_output(True)
            print(f"Llevamos {cont_entradas} entradas de {limite} ({round(100*cont_entradas/limite, 2)}%)")
            for ttkn in dic_ngramas:
                print(f"- {len(dic_ngramas[ttkn])} n-gramas recolectados del tipo {K.TTKN_DESC[ttkn]}")
    cont_entradas += 1
salta = limite
limite += incremento

Llevamos 230000 entradas de 230000 (100.0%)
- 915608 n-gramas recolectados del tipo Letras
- 3733 n-gramas recolectados del tipo Separadores
- 17216 n-gramas recolectados del tipo Dígitos
- 602 n-gramas recolectados del tipo Símbolos
- 18109 n-gramas recolectados del tipo Desconocidos
- 6 n-gramas recolectados del tipo Especiales


# Contabilizacion de monogramas

In [41]:
cont_monogramas = {x:{} for x in dic_ngramas}
apar_monogramas = {x:0 for x in dic_ngramas}

for ttkn in dic_ngramas:
    print(f"Empezando tipo {K.TTKN_DESC[ttkn]}...")
    for n_grama in dic_ngramas[ttkn]:
        for caracter in n_grama:
            invent._dict_counter(ttkn, caracter, cont_monogramas)
            apar_monogramas[ttkn] += 1
print("Terminado...")
for ttkn in apar_monogramas:
    print(f"{apar_monogramas[ttkn]} para el tipo {K.TTKN_DESC[ttkn]}")

Empezando tipo Letras...
Empezando tipo Separadores...
Empezando tipo Dígitos...
Empezando tipo Símbolos...
Empezando tipo Desconocidos...
Empezando tipo Especiales...
Terminado...
7635295 para el tipo Letras
24294 para el tipo Separadores
86995 para el tipo Dígitos
1975 para el tipo Símbolos
18109 para el tipo Desconocidos
57 para el tipo Especiales


probabilidad de monogramas

In [47]:
prob_monogramas = {x:{} for x in dic_ngramas}
for ttkn in cont_monogramas:
    for monograma in cont_monogramas[ttkn]:
        prob_monogramas[ttkn][monograma] = cont_monogramas[ttkn][monograma] / apar_monogramas[ttkn]
for ttkn in prob_monogramas:
    maximos = [monograma.replace("\n", " ") for monograma in sorted(prob_monogramas[ttkn], key=lambda x: prob_monogramas[ttkn][x], reverse=True)[:10]]
    print(f"Para {K.TTKN_DESC[ttkn].rjust(15)}: {', '.join([f'<{x}>: {round(prob_monogramas[ttkn][x], 7)}' for x in maximos])}")

Para          Letras: <a>: 0.1229755, <e>: 0.0906358, <i>: 0.0841251, <o>: 0.0747421, <r>: 0.0731348, <n>: 0.0675889, <s>: 0.0659625, <t>: 0.0523367, <l>: 0.0498435, <c>: 0.0423973
Para     Separadores: < >: 0.5441261, <.>: 0.1494196, <">: 0.0592327, <,>: 0.04816, <->: 0.0365111, <:>: 0.0324772, <'>: 0.0314893, < >: 0.5441261, <_>: 0.017494, <;>: 0.0172882
Para         Dígitos: <0>: 0.1493649, <1>: 0.1299385, <2>: 0.1091327, <3>: 0.0940974, <4>: 0.0925685, <5>: 0.0907523, <6>: 0.086959, <7>: 0.0840738, <8>: 0.0817633, <9>: 0.0813495
Para        Símbolos: <}>: 0.2794937, <)>: 0.1017722, <{>: 0.0835443, <&>: 0.0789873, <(>: 0.0724051, <]>: 0.0551899, <!>: 0.0516456, <|>: 0.0450633, <?>: 0.038481, <[>: 0.0374684
Para    Desconocidos: <ɾ>: 5.52e-05, <ˈ>: 5.52e-05, <ː>: 5.52e-05, <ž>: 5.52e-05, <č>: 5.52e-05, <​>: 5.52e-05, <«>: 5.52e-05, <»>: 5.52e-05, <“>: 5.52e-05, <”>: 5.52e-05
Para      Especiales: <	>: 1.0


## Combinatorias

In [ ]:
fragmentos = {ttkn: {} for ttkn in dic_ngramas}
fragmentos_n_grama = {ttkn: {} for ttkn in dic_ngramas}
print(f"Fragmentando ...".rjust(50) + "".join(["|"]*100))
for ttkn in dic_ngramas:
    print(f"Fragmentando {len(dic_ngramas[ttkn])} n-gramas de {K.TTKN_DESC[ttkn]}".rjust(50), end="")
    cada = 1+ len(dic_ngramas[ttkn]) // 100
    cont = 0
    for n_grama in dic_ngramas[ttkn]:
        for l_n in range(2, len(n_grama) - 1):
            for i_ini in range(len(n_grama) - l_n):
                if n_grama[i_ini:i_ini+l_n] in fragmentos[ttkn]:
                    fragmentos[ttkn][n_grama[i_ini:i_ini+l_n]] += 1
                    if n_grama in fragmentos_n_grama[ttkn][n_grama[i_ini:i_ini+l_n]]:
                        fragmentos_n_grama[ttkn][n_grama[i_ini:i_ini+l_n]][n_grama] += 1
                    else:
                        fragmentos_n_grama[ttkn][n_grama[i_ini:i_ini+l_n]][n_grama] = 1
                else:
                    fragmentos[ttkn][n_grama[i_ini:i_ini+l_n]] = 1
                    fragmentos_n_grama[ttkn][n_grama[i_ini:i_ini+l_n]] = {n_grama: 1}
        cont += 1
        if cont % cada == 0:
            print(".", end="")
    print("!")

                                  Fragmentando ...||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
            Fragmentando 915608 n-gramas de Letras...................................................................................................!
         Fragmentando 3733 n-gramas de Separadores..................................................................................................!
            Fragmentando 17216 n-gramas de Dígitos...................................................................................................!
             Fragmentando 602 n-gramas de Símbolos......................................................................................!
       Fragmentando 18109 n-gramas de Desconocidos...................................................................................................!
             Fragmentando 6 n-gramas de Especiales......!


## Fragmentos óptimos

In [ ]:
acum_frag_n_grama = {ttkn: sum([len(fragmentos_n_grama[ttkn][x]) for x in fragmentos_n_grama[ttkn]]) for ttkn in fragmentos_n_grama}
acum_fragmentos = {ttkn: sum([fragmentos[ttkn][x] for x in fragmentos[ttkn]]) for ttkn in fragmentos}

In [ ]:
fragmentos_usados = {ttkn: {} for ttkn in fragmentos}
excedentes = {ttkn: {} for ttkn in fragmentos}
fragmentos_evaluados = {}
print(f"Evaluando fragmentos sobre tokens ...".rjust(50) + "".join(["|"]*100))
for ttkn in dic_ngramas:
    print(f"Ordenando {len(fragmentos[ttkn])} fragmentos de {K.TTKN_DESC[ttkn]}...")
    fragmentos_evaluados[ttkn] = sorted(
        fragmentos[ttkn],
        key=lambda x: len(x) * (fragmentos[ttkn][x] / acum_fragmentos[ttkn]) * (len(fragmentos_n_grama[ttkn][x]) /acum_frag_n_grama[ttkn] ),
        reverse=True
    )
    cada = 1+ len(dic_ngramas[ttkn]) // 100
    print(f"Evaluando {len(dic_ngramas[ttkn])} n-gramas de {K.TTKN_DESC[ttkn]} cada {cada} un punto.".rjust(50), end="")
    cont = 0
    for n_grama in dic_ngramas[ttkn]:
        if isinstance(n_grama, str):
            invent.explota_fragmento(n_grama, fragmentos_usados[ttkn], excedentes[ttkn], fragmentos_evaluados[ttkn])
        cont += 1
        if cont % cada == 0:
            print(".", end="")
    print("!")

             Evaluando fragmentos sobre tokens ...||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
Ordenando 4010068 fragmentos de Letras...
Evaluando 915608 n-gramas de Letras cada 9157 un punto.

In [ ]:
fragmentos_usados_path = r"D:\datos\tokens_usados.pkl"
excedentes_path = r"D:\datos\tokens_excedentes.pkl"
fragmentos_evaluados_path = r"D:\datos\tokens_evaluados.pkl"
print(f"Guardando {fragmentos_usados_path}")
with open(fragmentos_usados_path, "wb") as file:
    pickle.dump(fragmentos_usados)
print(f"Guardando {excedentes_path}")
with open(excedentes_path, "wb") as file:
    pickle.dump(excedentes)
print(f"Guardando {fragmentos_evaluados_path}")
with open(fragmentos_evaluados_path, "wb") as file:
    pickle.dump(fragmentos_evaluados)

In [ ]:
tokens = {K.TTKN_SIS:token_id for token_id in K.MAIN_TOKEN_DICT} + {ttkn: {} for ttkn in fragmentos}
token_id = 10
for ttkn in fragmentos_evaluados:
    token_id += 1000 * ttkn
    for fragmento in fragmentos_evaluados[ttkn]:
        if fragmento in fragmentos_usados[ttkn]:
            tokens[ttkn][fragmento] = token_id
            token_id += 1
    for excedente in excedentes[ttkn]:
        tokens[ttkn][excedente] = token_id
        token_id += 1


# OLD code

In [40]:
num_ngramas = len(n_gramas)
acum_ngramas = 0
min_apariciones = 10000000
min_ngrama = ""
max_apariciones = 0
max_ngrama = ""
for n_grama in n_gramas:
    acum_ngramas += n_gramas[n_grama]
    if n_gramas[n_grama] < min_apariciones:
        min_apariciones = n_gramas[n_grama]
        min_ngrama = n_grama
    if n_gramas[n_grama] > max_apariciones:
        max_apariciones = n_gramas[n_grama]
        max_ngrama = n_grama
print(f"Tenemos un total de {acum_ngramas} apariciones de {num_ngramas} n-gramas")
print(f"Una media de {round(acum_ngramas/num_ngramas)} apariciones/n-grama")
print(f"El n-grama que menos veces aparece es '{min_ngrama}' con {min_apariciones} veces")
print(f"El n-grama que MÁS veces aparece es '{max_ngrama}' con {max_apariciones} veces")

Tenemos un total de 89269666 apariciones de 589563 n-gramas
Una media de 151 apariciones/n-grama
El n-grama que menos veces aparece es 'hebestreit' con 1 veces
El n-grama que MÁS veces aparece es ' ' con 35692569 veces


In [41]:
prob_ngramas = {}
pond_len_ngrama = {}
acum_pond_ngramas = 0
for n_grama in n_gramas:
    prob_ngramas[n_grama] = n_gramas[n_grama] / num_ngramas
    pond_len_ngrama[n_grama] = len(n_grama) * n_gramas[n_grama]
    acum_pond_ngramas += pond_len_ngrama[n_grama]
prob_pond_ngramas = {}
for n_grama in pond_len_ngrama:
    prob_pond_ngramas[n_grama] = pond_len_ngrama[n_grama] / acum_pond_ngramas

In [43]:
cont = 0
for n_grama in sorted(prob_pond_ngramas, key=lambda x:prob_pond_ngramas[x], reverse=True ):
    p_ngrama = n_grama.replace("\n", "¬")
    print(f"Ngrama '{p_ngrama}'".rjust(20), end="")
    cont += 1
    if cont % 6 == 0:
        print()
    if cont > 100:
        break

    Ngrama '<|CAP|>'          Ngrama ' '         Ngrama 'de'  Ngrama '<|UPPER|>'         Ngrama 'la'         Ngrama ', '
         Ngrama 'en'         Ngrama 'el'        Ngrama 'que'        Ngrama ' , '        Ngrama 'los'        Ngrama 'del'
         Ngrama '. '        Ngrama 'por'        Ngrama 'con'        Ngrama 'una'          Ngrama 'y'        Ngrama 'las'
       Ngrama 'para'         Ngrama 'se'       Ngrama 'como'         Ngrama 'un'          Ngrama 'a'         Ngrama 'su'
        Ngrama 'fue'    Ngrama 'también'        Ngrama ' . '         Ngrama 'es'         Ngrama 'al'         Ngrama '  '
      Ngrama 'entre'    Ngrama 'durante'        Ngrama 'más'    Ngrama 'después'          Ngrama '('        Ngrama '.  '
        Ngrama 'sus'          Ngrama ')'    Ngrama 'primera'      Ngrama 'desde'      Ngrama 'hasta'      Ngrama 'sobre'
       Ngrama 'este'  Ngrama 'población'     Ngrama 'ciudad'      Ngrama 'parte'     Ngrama 'cuando'      Ngrama 'donde'
Ngrama 'universidad'         Ngr

In [48]:
max_len = 0
max_len_ngrama = ""
for n_grama in n_gramas:
    if len(n_grama) > max_len:
        max_len = len(n_grama)
        max_len_ngrama = n_grama
print(f"El n-grama más largo tiene {max_len} caracteres: '{max_len_ngrama}'")
cont = 0
for n_grama in sorted(n_gramas, key=lambda x: len(x)+prob_ngramas[x], reverse=True):
    p_ngrama = n_grama.replace("\n", "¬")
    print(f"{str(len(n_grama)).rjust(3)} <|{p_ngrama}|>")
    cont += 1
    if cont > 20:
        break

El n-grama más largo tiene 217 caracteres: '                                                                                                                                                                                                                         '
217 <|                                                                                                                                                                                                                         |>
209 <|                                                                                                                                                                                                                 |>
200 <|                                                                                                                                                                                                        |>
186 <|.                                                                             

# Fragmentos
Contamos trozos de n-grama distintos

In [146]:
limite = 50 # Máximo número de caracteres admitidos
fragmentos = {}
fragmento_n_grama = {}
proto_fragmentos = []
cont = 0
print(f"Fragmentando {len(n_gramas)} n-gramas")
for n_grama in n_gramas:
    pre_fragmento = "" + n_grama
    while len(pre_fragmento) > limite:
        if pre_fragmento[:limite] not in proto_fragmentos:
            proto_fragmentos.append(pre_fragmento[:limite])
        pre_fragmento = pre_fragmento[limite:]
    if pre_fragmento > "" and pre_fragmento not in proto_fragmentos:
        proto_fragmentos.append(pre_fragmento)
    cont += 1
    if cont % 10000 == 0:
        clear_output()
        print(f"Vamos por el n_grama {cont} de {len(n_gramas)}")
print(f"... troceando {len(proto_fragmentos)} n-gramas")
cont = 0
for n_grama in proto_fragmentos:
    if cont == 0:
        print(f"Fragmentados {cont} n-gramas de {len(proto_fragmentos)}")
    for l_n in range(2, len(n_grama) - 1):
        for i_ini in range(len(n_grama) - l_n):
            if n_grama[i_ini:i_ini+l_n] in fragmentos:
                fragmentos[n_grama[i_ini:i_ini+l_n]] += 1
                if n_grama in fragmento_n_grama[n_grama[i_ini:i_ini+l_n]]:
                    fragmento_n_grama[n_grama[i_ini:i_ini+l_n]][n_grama] += 1
                else:
                    fragmento_n_grama[n_grama[i_ini:i_ini+l_n]][n_grama] = 1
            else:
                fragmentos[n_grama[i_ini:i_ini+l_n]] = 1
                fragmento_n_grama[n_grama[i_ini:i_ini+l_n]] = {n_grama: 1}
    cont += 1
    if cont % 10000 == 0:
        clear_output()
        print(f"Vamos por el n_grama {cont} de {len(proto_fragmentos)}")
del proto_fragmentos
print(f"Generados {len(fragmentos)} fragmentos de n-grama")

Vamos por el n_grama 580000 de 589563
Generados 2545350 fragmentos de n-grama


In [147]:
acum_fragmentos_in_ngramas = 0
cont_fragmentos_in_ngramas = 0
max_fragmentos_in_ngramas = 0
max_fragmentos_in_ngramas_fragmento = ""
for fragmento in fragmento_n_grama:
    acum_fragmentos_in_ngramas += len(fragmento_n_grama[fragmento])
    cont_fragmentos_in_ngramas += sum([fragmento_n_grama[fragmento][x] for x in fragmento_n_grama[fragmento]])
    if len(fragmento_n_grama[fragmento]) > max_fragmentos_in_ngramas:
        max_fragmentos_in_ngramas = len(fragmento_n_grama[fragmento])
        max_fragmentos_in_ngramas_fragmento = fragmento
print(f"Tenemos {len(fragmentos)} fragmentos con un total de {cont_fragmentos_in_ngramas} apariciones")
print(f"Hay {acum_fragmentos_in_ngramas} n-gramas con {round(acum_fragmentos_in_ngramas / len(fragmentos), 2)} n-gramas por fragmento de media")
print(f"El fragmento '{max_fragmentos_in_ngramas_fragmento}' es el que aparece en más n_gramas con {max_fragmentos_in_ngramas} n-gramas")

Tenemos 2545350 fragmentos con un total de 14698314 apariciones
Hay 14621202 n-gramas con 5.74 n-gramas por fragmento de media
El fragmento 'ar' es el que aparece en más n_gramas con 63612 n-gramas


In [148]:
prob_fragmentos = {}
for fragmento in fragmento_n_grama:
    prob_fragmentos[fragmento] = sum([fragmento_n_grama[fragmento][x] for x in fragmento_n_grama[fragmento]]) / cont_fragmentos_in_ngramas

# Fragmentos usados
Contamos los fragmentos de n-grama para ver su relación entre relevancia e información
- Se trata de ordenar los fragmentos según tengan más información y se repitan más veces

In [149]:
minima_mente = min([
    len(fragmento_n_grama["mente"])/acum_fragmentos_in_ngramas if "mente" in fragmento_n_grama else 0.0,
    len(fragmento_n_grama["ndo"])/acum_fragmentos_in_ngramas if "ndo" in fragmento_n_grama else 0.0,
    len(fragmento_n_grama["ido"])/acum_fragmentos_in_ngramas if "ido" in fragmento_n_grama else 0.0
])
print("Minima-mente está en {minima_mente}")
fragmentos_ok = [x for x in fragmentos if len(fragmento_n_grama[x])/acum_fragmentos_in_ngramas >= minima_mente]
print(f"Tenemos {len(fragmentos_ok)} fragmentos seleccionados")
cont = 0
for fragmento in sorted(fragmentos_ok, key=lambda x: len(x) + prob_fragmentos[x] + len(fragmento_n_grama[x])/acum_fragmentos_in_ngramas, reverse=True):
    p_frag = fragmento.replace("\n", "¬")
    print(f"'{p_frag}'".ljust(20), end=" ")
    cont += 1
    if cont % 8 == 0:
        print()
    if cont > 400:
        break

Minima-mente está en {minima_mente}
Tenemos 9926 fragmentos seleccionados
'alizació'           'ticament'           'ificació'           'presenta'           'american'           'tivament'           'acionali'           'structur'           
'icament'            'lizació'            'ográfic'            'alizaci'            'acional'            'ificaci'            'electro'            'cionali'            
'amiento'            'present'            'ificado'            'ecciona'            'adament'            'ológica'            'alizado'            'abilida'            
'ticamen'            'ntement'            'ivament'            'ficació'            'constru'            'resenta'            'america'            'ferenci'            
'tacione'            'rizació'            'ministr'            'termina'            'namient'            'naciona'            'osament'            'merican'            
'cionale'            'olucion'            'cimient'            'erencia'         

In [150]:
def exprime_fragmento(n_grama, fragmentos_usados, excedentes, fragmentos_evaluados):
    cont_excedentes = 1
    cont_sub_excedentes = 0
    for fragmento in fragmentos_evaluados:
        if len(fragmento) > len(n_grama):
            continue
        elif fragmento == n_grama:
            cont_excedentes = 0  # contador a cero
            if fragmento in fragmentos_usados:
                fragmentos_usados[fragmento] += 1
            else:
                fragmentos_usados[fragmento] = 1
        elif fragmento in n_grama:
            cont_excedentes = 0
            if fragmento in fragmentos_usados:
                fragmentos_usados[fragmento] += 1
            else:
                fragmentos_usados[fragmento] = 1
            for x in n_grama.split(fragmento):
                if x and x > "":
                    cont_sub_excedentes += exprime_fragmento(x, fragmentos_usados, excedentes, fragmentos_evaluados)
            break
    if cont_excedentes > 0:
        if n_grama in excedentes:
            excedentes[n_grama] += 1
        else:
            excedentes[n_grama] = 1
    return cont_excedentes + cont_sub_excedentes

In [151]:
fragmentos_evaluados = sorted(fragmentos_ok, key=lambda x: len(x) + prob_fragmentos[x] + len(fragmento_n_grama[x])/acum_fragmentos_in_ngramas, reverse=True)
fragmentos_usados = {}
excedentes = {}
cont_usados = 0
cont_excedentes = 0
cont_existentes = 0
cont = 0
for n_grama in n_gramas:
    cont_excedentes += exprime_fragmento(n_grama, fragmentos_usados, excedentes, fragmentos_evaluados)
    cont += 1
    if cont % 10000 == 0:
        clear_output()
        print(f"Llevamos {cont} de {len(n_gramas)}")
        print(f"{len(fragmentos_usados)} usados y {cont_excedentes} generados...")

Llevamos 580000 de 589563
9862 usados y 643460 generados...


In [152]:
cont = 0
for fragmento in sorted(fragmentos_usados, key=lambda x:fragmentos_usados[x], reverse=True):
    p_frag = fragmento.replace("\n", "¬")
    print(f"<|{p_frag}|>".ljust(15), end=" ")
    cont += 1
    if cont % 10 == 0:
        print()
    if cont > 200:
        break

<|os|>          <|as|>          <|es|>          <|re|>          <|an|>          <|in|>          <|on|>          <|us|>          <|en|>          <|al|>          
<|co|>          <|se|>          <|er|>          <|ta|>          <|ar|>          <|do|>          <|de|>          <|ma|>          <|la|>          <|le|>          
<|ca|>          <|ra|>          <|is|>          <|su|>          <|na|>          <|te|>          <|ba|>          <|ro|>          <|da|>          <|ch|>          
<|or|>          <|to|>          <|sa|>          <|ón|>          <|ne|>          <|lo|>          <|um|>          <|mo|>          <|di|>          <|ka|>          
<|ad|>          <|el|>          <|ur|>          <|án|>          <|no|>          <|li|>          <|ri|>          <|me|>          <|bo|>          <|ex|>          
<|ti|>          <|pa|>          <|po|>          <|ni|>          <|si|>          <|so|>          <|ha|>          <|gu|>          <|des|>         <|ía|>          
<|at|>          <|cu|>          <|

In [153]:
cont = 0
for fragmento in sorted(excedentes, key=lambda x:len(x), reverse=True):
    p_frag = fragmento.replace("\n", "¬")
    if fragmento in fragmentos_usados:
        p_frag = " EXISTENTE: " + p_frag
    print(f"<|{p_frag}|>".ljust(15), end=" ")
    cont += 1
    if cont % 10 == 0:
        print()
    if cont > 200:
        break

<|,\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;|> <|\;\;\;\;\;\;\;\;\;\;\;\;|> <|&&&&&&&&&&&&&|> <|&&&&&&&&&&&&|> <|&&&&&&&&&&&|> <|&&&&&&&&&&|>  <|----------|>  <|<|UPPER|>|>   <|&&&&&&&&&|>   <| _______ |>   
<| ______ |>    <|}}})}}}}|>    <|&&&&&&&&|>    <|<|CAP|>|>     <|wjbqfvz|>     <| ' ' ' |>     <| \ +\ \|>     <|?!!!??!|>     <| + + + |>     <| ______|>     
<|\\\ \ \|>     <| ++ / |>      <| ' ' '|>      <|' / ' |>      <|\\:\\\|>      <| ' '- |>      <| \ \ \|>      <|}}}}}}|>      <|?????)|>      <|}}}}}{|>      
<|(¿?!¡)|>      <|}})}}}|>      <|}}|}}}|>      <|}}]]]}|>      <|})}}}}|>      <|[????]|>      <|---- /|>      <|---- |>       <|&&&&&|>       <| / ; |>       
<|{}}&{|>       <| '+' |>       <|}}}{(|>       <|}}}}}|>       <|\\:\\|>       <| +/- |>       <|}}{({|>       <|})}{{|>       <|}})}{|>       <|\ +\ |>       
<|}}}~{|>       <|\ \ \|>       <| \ \ |>       <

In [154]:
fragmentos_re_evaluados = sorted([x for x in fragmentos if fragmento not in fragmentos_usados], key=lambda x: len(x) + prob_fragmentos[x] + len(fragmento_n_grama[x])/acum_fragmentos_in_ngramas, reverse=True)
print(f"Repetimos con {len(fragmentos_re_evaluados)} restos")
fragmentos_reusados = {}
nuevos_excedentes = {}
cont_excedentes = 0
for n_grama in excedentes:
    cont_excedentes += exprime_fragmento(n_grama, fragmentos_reusados, nuevos_excedentes, fragmentos_re_evaluados)
    cont += 1
    if cont % 10000 == 0:
        clear_output()
        print(f"Llevamos {cont} de {len(excedentes)}")
        print(f"{len(fragmentos_reusados)} usados y {cont_excedentes} generados...")

Repetimos con 2545350 restos


In [155]:
print(len(fragmentos_reusados))
cont = 0
for fragmento in sorted(fragmentos_reusados, key=lambda x:len(x), reverse=True):
    p_frag = fragmento.replace("\n", "¬")
    if fragmento in fragmentos_usados:
        p_frag = " EXISTENTE: " + p_frag
    print(f"<|{p_frag}|>".ljust(15), end=" ")
    cont += 1
    if cont % 10 == 0:
        print()
    if cont > 200:
        break

848
<|\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;|> <|\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;|> <|;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;|> <|\;\;\;\;\;\;\;\;\;\;\;\;|> <|;\;\;\;\;\;\;\;\;\;\;\;|> <|&&&&&&&&&&&|> <|&&&&&&&&&&|>  <|----------|>  <|&&&&&&&&&|>   <|---------|>   
<|&&&&&&&&|>    <|<|UPPER|>     <|wjbqfvz|>     <| + + + |>     <| ______|>     <|\\\ \ \|>     <|&&&&&&&|>     <|wjbqfv|>      <| ' ' '|>      <| + + +|>      
<|______|>      <|}}})}}|>      <|\\\ \ |>      <|}})}}}|>      <|---- /|>      <|<|CAP|>       <|---- |>       <|&&&&&|>       <| ' ' |>       <|\ +\ |>       
<| \ +\|>       <|\ \ \|>       <| \ \ |>       <|,\;\;|>       <| ''' |>       <|?!!!?|>       <| + + |>       <|hxjvx|>       <|.¬¬¬¬|>       <|'''''|>       
<|yüsüp|>       <|}})}}|>       <|yóújf|>       <|})}}}|>       <|}}})}|>       <|----|>        <| '' |>        <|&&&&|>        <|/ ; |>        <|}}}}|>        
<| ++ |>        <|\;\;|>        <|,\;\|>        <|' / |>        

In [156]:
print(len(nuevos_excedentes))
cont = 0
for fragmento in sorted(nuevos_excedentes, key=lambda x:len(x), reverse=True):
    p_frag = fragmento.replace("\n", "¬")
    if fragmento in fragmentos_usados:
        p_frag = " EXISTENTE: " + p_frag
    print(f"<|{p_frag}|>".ljust(15), end=" ")
    cont += 1
    if cont % 10 == 0:
        print()
    if cont > 200:
        break

5988
<|(?)|>         <|[<]|>         <|<)>|>         <|($)|>         <|{{{|>         <|)]·|>         <|)](|>         <|(€)|>         <|)>(|>         <|)()|>         
<|(%)|>         <|(|||>         <|(>)|>         <|)({|>         <|(!)|>         <|)([|>         <|]([|>         <|)·(|>         <|>{{|>         <|¡¡¡|>         
<|'/'|>         <|(~)|>         <|&<<|>         <|{<}|>         <|···|>         <|)(?|>         <|({{|>         <|{{<|>         <|)(||>         <|)&{|>         
<|]~[|>         <|(#)|>         <||>|>          <|%}|>          <|}@|>          <|){|>          <|%)|>          <|(¡|>          <|!)|>          <|:+|>          
<|()|>          <|({|>          <|)&|>          <|[{|>          <|]?|>          <|&{|>          <|?)|>          <|)]|>          <|<<|>          <|($|>          
<||||>          <||{|>          <|)(|>          <|º)|>          <|%%|>          <|[(|>          <|¬/|>          <|](|>          <|·(|>          <|[¡|>          
<|([|>          <|{{|>       

In [141]:
pre_tokens_id = {}
token_id = 1
for fragmento in fragmentos_evaluados:
    if fragmento in fragmentos_usados:
        pre_tokens_id[fragmento] = token_id
        token_id += 1
for fragmento in fragmentos_re_evaluados:
    if fragmento in fragmentos_reusados:
        pre_tokens_id[fragmento] = token_id
        token_id += 1
for fragmento in nuevos_excedentes:
    pre_tokens_id[fragmento] = token_id
    token_id += 1
print(f"Tenemos {len(pre_tokens_id)} pre tokens")

Tenemos 16694 pre tokens


In [142]:
cont = 0
for token in pre_tokens_id:
    p_token = token.replace("\n", "¬")
    print(f"<|{p_token}|>".ljust(15), end=" ")
    cont += 1
    if cont % 10 == 0:
        print()
    if cont > 300:
        break

<|alizació|>    <|ticament|>    <|ificació|>    <|presenta|>    <|american|>    <|tivament|>    <|acionali|>    <|structur|>    <|icament|>     <|lizació|>     
<|ográfic|>     <|alizaci|>     <|acional|>     <|ificaci|>     <|electro|>     <|cionali|>     <|amiento|>     <|present|>     <|ificado|>     <|ecciona|>     
<|adament|>     <|ológica|>     <|alizado|>     <|abilida|>     <|ticamen|>     <|ntement|>     <|ivament|>     <|ficació|>     <|constru|>     <|resenta|>     
<|america|>     <|ferenci|>     <|tacione|>     <|rizació|>     <|ministr|>     <|termina|>     <|namient|>     <|naciona|>     <|osament|>     <|merican|>     
<|cionale|>     <|olucion|>     <|cimient|>     <|erencia|>     <|racione|>     <|estruct|>     <|ológico|>     <|blement|>     <|cionist|>     <|mentari|>     
<|onaliza|>     <|organiz|>     <|ométric|>     <|structu|>     <|interpr|>     <|entemen|>     <|histori|>     <|bacteri|>     <|acione|>      <|amient|>      
<|izació|>      <|contra|>      <|